In [2]:
#%pip install lxml
import os
import s3fs
import pandas as pd
import xml.etree.ElementTree as ET
#from lxml import etree

os.environ["AWS_ACCESS_KEY_ID"] = 'GM7L3CZGK8C9Z4SZNE0W'
os.environ["AWS_SECRET_ACCESS_KEY"] = 'FEvrbDToCE0+DSM2Q4cwvO7gLMFMSMZ66p2wbwXM'
os.environ["AWS_SESSION_TOKEN"] = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiJHTTdMM0NaR0s4QzlaNFNaTkUwVyIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzcwOTk4NzM2LCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6Imx1Y2FzLmN1bXVuZWxAZW5zYWUuZnIiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZXhwIjoxNzcxODczODUxLCJmYW1pbHlfbmFtZSI6IkN1bXVuZWwiLCJnaXZlbl9uYW1lIjoiTHVjYXMiLCJncm91cHMiOlsiVVNFUl9PTllYSUEiLCJzdGF0YXBwLXNlZ21lZGljIl0sImlhdCI6MTc3MTI2OTA1MSwiaXNzIjoiaHR0cHM6Ly9hdXRoLmxhYi5zc3BjbG91ZC5mci9hdXRoL3JlYWxtcy9zc3BjbG91ZCIsImp0aSI6Im9ucnRydDphYzQ3MTIxMS03YjE2LWRmNGMtYzA5ZS1lNTUzMmZmOWExNWIiLCJuYW1lIjoiTHVjYXMgQ3VtdW5lbCIsInBvbGljeSI6InN0c29ubHkiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJsYWIiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiIsImRlZmF1bHQtcm9sZXMtc3NwY2xvdWQiXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1zc3BjbG91ZCJdLCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGdyb3VwcyBlbWFpbCIsInNpZCI6IjgxNDA0Yzg0LTk2YjAtZGFjMS03NDEzLTBjZmZjMzQ2MWVlZiIsInN1YiI6ImUyZDc4NjRjLTcwMzItNDI0ZC04OTA2LWU0ZjhiNDFjYzAwMyIsInR5cCI6IkJlYXJlciJ9.qWNp1S72FCU28Xg1goyuRh6oj7FfNNZ2M52AEiGM8CioNLpTl-Xd-10qGLsA_qwRjgMWXUSR3cYCnKNXXd-EGQ'
os.environ["AWS_DEFAULT_REGION"] = 'us-east-1'
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key = os.environ["AWS_ACCESS_KEY_ID"], 
    secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
    token = os.environ["AWS_SESSION_TOKEN"])
with fs.open("s3://lab/pid_name.csv") as f:
   pid = pd.read_csv(f)
with fs.open("s3://lab/conf_net.parquet") as f:
    conf = pd.read_parquet(f)

with fs.open("s3://lab/art_net.parquet") as f:
    art = pd.read_parquet(f)




In [ ]:
name_conf = pd.DataFrame(
    [a for authors in conf["authors"] for a in authors]

)

name_art = pd.DataFrame(
[a for authors in art["authors"] for a in authors]
)

In [ ]:
merge = pid.merge(name_art, on="name",how="left")

In [ ]:
import requests
import time
from bs4 import BeautifulSoup


headers = {
    "User-Agent": "Mozilla/5.0"
}

results = []

# ----- Loop through links -----
for url in pid["pid"][:1_000_000]:
    print(f"Scraping: {url}")
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, "html.parser")

        affiliations = []

        # Extract affiliation names
        for block in soup.select('li[itemprop="affiliation"]'):
            name = block.select_one('[itemprop="name"]')
            
            if name:
                affiliations.append(name.get_text(strip=True))

        # Store results
        results.append({
            "profile": url,
            "affiliations": affiliations
        })

    except Exception as e:
        print(f"Error scraping {url}: {e}")

    # ----- Cooldown (important!) -----
    time.sleep(0.5)   # change delay if needed


Rutgers University, Department of Psychology, New Brunswick, NJ, USA
Massachusetts Institute of Technology, Department of Brain and Cognitive Sciences, Cambridge, MA, USA
University of California Irvine, Department of Cognitive Sciences, CA, USA


In [ ]:
import asyncio
import random
import aiohttp
from bs4 import BeautifulSoup
import pandas as pd
import s3fs
import nest_asyncio

nest_asyncio.apply()  # Jupyter-safe

# ---- Config ----
fs = s3fs.S3FileSystem()
S3_PREFIX = "s3://lab/orcids/"
BATCH_SIZE = 100_000
CONCURRENCY = 8

result = []
file_counter = 0
semaphore = asyncio.Semaphore(CONCURRENCY)

# ---- Scraper ----
async def scrape(session, url):
    global result, file_counter

    async with semaphore:
        try:
            async with session.get(url) as response:
                html = await response.text()
                soup = BeautifulSoup(html, "html.parser")

                affiliations = [
                    a.get_text(strip=True)
                    for a in soup.select('li[itemprop="affiliation"] [itemprop="name"]')
                ]

                result.append({"url": url, "affiliations": affiliations})
                print(url, affiliations)

                # ---- Flush every 100k rows ----
                if len(result) >= BATCH_SIZE:
                    df_batch = pd.DataFrame(result)
                    file_path = f"{S3_PREFIX}affil_{file_counter}.csv"
                    with fs.open(file_path, "wb") as f:
                        df_batch.to_csv(f, index=False)
                    print(f"Flushed batch #{file_counter} to {file_path}")
                    result.clear()
                    file_counter += 1

                await asyncio.sleep(random.uniform(0.5, 1.2))

        except Exception as e:
            print("Error:", url, e)

# ---- Main runner ----
async def main(pid_list):
    async with aiohttp.ClientSession() as session:
        for url in pid_list:
            await scrape(session, url)

# ---- Run ----
await main(pid["pid"][1_000_000:])

# ---- Flush remaining rows ----
if result:
    df_batch = pd.DataFrame(result)
    file_path = f"{S3_PREFIX}affil_{file_counter}.csv"
    with fs.open(file_path, "wb") as f:
        df_batch.to_csv(f, index=False)
    print(f"Flushed final batch #{file_counter} to {file_path}")


In [ ]:
df = pd.concat(
    [art.explode("authors")["authors"],
     conf.explode("authors")["authors"]],
    ignore_index=True
)

df = df.apply(pd.Series)[['name', 'orcid']]
df

In [ ]:
browser = await p.chromium.launch(
    headless=True,
    args=["--no-sandbox", "--disable-setuid-sandbox"]
)


TargetClosedError: BrowserType.launch: Target page, context or browser has been closed
Browser logs:

<launching> /home/onyxia/.cache/ms-playwright/chromium_headless_shell-1208/chrome-headless-shell-linux64/chrome-headless-shell --disable-field-trial-config --disable-background-networking --disable-background-timer-throttling --disable-backgrounding-occluded-windows --disable-back-forward-cache --disable-breakpad --disable-client-side-phishing-detection --disable-component-extensions-with-background-pages --disable-component-update --no-default-browser-check --disable-default-apps --disable-dev-shm-usage --disable-extensions --disable-features=AvoidUnnecessaryBeforeUnloadCheckSync,BoundaryEventDispatchTracksNodeRemoval,DestroyProfileOnBrowserClose,DialMediaRouteProvider,GlobalMediaControls,HttpsUpgrades,LensOverlay,MediaRouter,PaintHolding,ThirdPartyStoragePartitioning,Translate,AutoDeElevate,RenderDocument,OptimizationHints --enable-features=CDPScreenshotNewSurface --allow-pre-commit-

In [16]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

URL = "https://orcid.org/0000-0002-2058-0693"

async def scrape_orcid_panels():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True, args=["--no-sandbox"])
        page = await browser.new_page()
        await page.goto(URL, timeout=60000)

        # Scroll to trigger lazy loading
        for _ in range(5):
            await page.evaluate("window.scrollBy(0, document.body.scrollHeight)")
            await page.wait_for_timeout(1000)

        await page.wait_for_selector("#activities")

        panels = await page.query_selector_all("orcid-panel")
        data = []
        for idx, panel in enumerate(panels):
            text = await panel.inner_text()
            if not text.strip():
                continue
            data.append({
                "panel_index": idx,
                "text": text
            })

        await browser.close()
        return data

# In Jupyter
df = pd.DataFrame(await scrape_orcid_panels())
df.head()


TargetClosedError: BrowserType.launch: Target page, context or browser has been closed
Browser logs:

<launching> /home/onyxia/.cache/ms-playwright/chromium_headless_shell-1208/chrome-headless-shell-linux64/chrome-headless-shell --disable-field-trial-config --disable-background-networking --disable-background-timer-throttling --disable-backgrounding-occluded-windows --disable-back-forward-cache --disable-breakpad --disable-client-side-phishing-detection --disable-component-extensions-with-background-pages --disable-component-update --no-default-browser-check --disable-default-apps --disable-dev-shm-usage --disable-extensions --disable-features=AvoidUnnecessaryBeforeUnloadCheckSync,BoundaryEventDispatchTracksNodeRemoval,DestroyProfileOnBrowserClose,DialMediaRouteProvider,GlobalMediaControls,HttpsUpgrades,LensOverlay,MediaRouter,PaintHolding,ThirdPartyStoragePartitioning,Translate,AutoDeElevate,RenderDocument,OptimizationHints --enable-features=CDPScreenshotNewSurface --allow-pre-commit-

In [ ]:
with fs.open(f"s3://lab/orcids.parquet", "wb") as f:
            df.to_parquet(f, index=False)

,title,journal,year,authors,dblp_uri,doi
0,Auswirkung der Digitalisierung auf die Systeml...,Elektrotech. Informationstechnik,2019,"{'name': 'Kai Schlabitz', 'orcid': None}",journals/ei/Schlabitz19,https://doi.org/10.1007/s00502-019-0687-y
1,EMF-Personenschutz: Neue Aspekte in der numeri...,Elektrotech. Informationstechnik,2020,"{'name': 'Richard berbacher', 'orcid': None}",journals/ei/UberbacherC20,https://doi.org/10.1007/s00502-020-00791-z
1,EMF-Personenschutz: Neue Aspekte in der numeri...,Elektrotech. Informationstechnik,2020,"{'name': 'Stefan Cecil', 'orcid': None}",journals/ei/UberbacherC20,https://doi.org/10.1007/s00502-020-00791-z
2,Zur Genesis der Forschungsstelle fr Integriert...,Elektrotech. Informationstechnik,2022,"{'name': 'Herbert Mang', 'orcid': None}",journals/ei/Mang22,https://doi.org/10.1007/s00502-022-01049-6
3,100 % erneuerbare Energie fr sterreichs Indust...,Elektrotech. Informationstechnik,2021,"{'name': 'Sophie Knttner', 'orcid': None}",journals/ei/KnottnerGDD21,https://doi.org/10.1007/s00502-021-00953-7
...,...,...,...,...,...,...
3992220,Analysis of projected hydrological behavior of...,Hydrology and Earth System Sciences,2012,"{'name': 'Rita Ley', 'orcid': None}",persons/CasperGGGHLR12,https://doi.org/10.5194/hess-16-409-2012
3992220,Analysis of projected hydrological behavior of...,Hydrology and Earth System Sciences,2012,"{'name': 'Andreas Rock', 'orcid': None}",persons/CasperGGGHLR12,https://doi.org/10.5194/hess-16-409-2012
3992221,Common Subexpression Identification in General...,"Technical Rep. UKSC 0060, IBM United Kingdom S...",1974,"{'name': 'Patrick A. V. Hall', 'orcid': None}",persons/Hall74,None
3992222,Interactive Support for Non-Programmers: The R...,"Research Report / RJ / IBM / San Jose, California",1974,"{'name': 'E. F. Codd', 'orcid': None}",persons/CoddD74,None
